In [1]:
import requests
import pandas as pd

import numpy as np

## Extract

In [2]:
# url for api

url = 'https://randomuser.me/api/'

num_of_users = '?results=100' # we can select the number of user to pull

url = url + num_of_users

print(url)

https://randomuser.me/api/?results=100


there are two major types of method in api: get and post

In [3]:
# hitting the api and storing the response

response = requests.get(url)


In [4]:
# if response is 200 then success, if 500 then internal error, 404 means not found
response

<Response [200]>

In [5]:
if response.status_code == 200: # to check if we are getting results
    data = response.json() # storing the result

In [6]:
data

{'results': [{'gender': 'female',
   'name': {'title': 'Mrs', 'first': 'Mattie', 'last': 'Reed'},
   'location': {'street': {'number': 1846, 'name': 'Blossom Hill Rd'},
    'city': 'Rockhampton',
    'state': 'New South Wales',
    'country': 'Australia',
    'postcode': 5310,
    'coordinates': {'latitude': '-85.0727', 'longitude': '156.8412'},
    'timezone': {'offset': '+10:00',
     'description': 'Eastern Australia, Guam, Vladivostok'}},
   'email': 'mattie.reed@example.com',
   'login': {'uuid': 'ab7f20b3-83f8-409e-a184-08957eca3b2e',
    'username': 'crazybear594',
    'password': 'macaroni',
    'salt': 'n9GcL4Qf',
    'md5': 'f92f78c00ab6275cb2be958e9adfce52',
    'sha1': '401f40d8c9814e50cea1ab5dae1b6678b9f64a9d',
    'sha256': 'f277ba6b65ef27b4492a44e33ef5402ad78489a9825e8e345b2ccfea722f4119'},
   'dob': {'date': '1976-12-16T04:08:53.186Z', 'age': 47},
   'registered': {'date': '2008-08-04T03:25:11.063Z', 'age': 16},
   'phone': '09-7627-1907',
   'cell': '0453-553-546',
   

In [7]:
df = pd.json_normalize(data) # decoding the json

In [8]:
df

,results,info.seed,info.results,info.page,info.version
0,"[{'gender': 'female', 'name': {'title': 'Mrs',...",a24adf7e8f73d697,100,1,1.4


In [9]:
for i in df['results']: # looking over each record within results column, it is nested json
    print(i)

[{'gender': 'female', 'name': {'title': 'Mrs', 'first': 'Mattie', 'last': 'Reed'}, 'location': {'street': {'number': 1846, 'name': 'Blossom Hill Rd'}, 'city': 'Rockhampton', 'state': 'New South Wales', 'country': 'Australia', 'postcode': 5310, 'coordinates': {'latitude': '-85.0727', 'longitude': '156.8412'}, 'timezone': {'offset': '+10:00', 'description': 'Eastern Australia, Guam, Vladivostok'}}, 'email': 'mattie.reed@example.com', 'login': {'uuid': 'ab7f20b3-83f8-409e-a184-08957eca3b2e', 'username': 'crazybear594', 'password': 'macaroni', 'salt': 'n9GcL4Qf', 'md5': 'f92f78c00ab6275cb2be958e9adfce52', 'sha1': '401f40d8c9814e50cea1ab5dae1b6678b9f64a9d', 'sha256': 'f277ba6b65ef27b4492a44e33ef5402ad78489a9825e8e345b2ccfea722f4119'}, 'dob': {'date': '1976-12-16T04:08:53.186Z', 'age': 47}, 'registered': {'date': '2008-08-04T03:25:11.063Z', 'age': 16}, 'phone': '09-7627-1907', 'cell': '0453-553-546', 'id': {'name': 'TFN', 'value': '082931467'}, 'picture': {'large': 'https://randomuser.me/api

In [10]:
# Create lists to store the extracted user data
first_name_ls = []
last_name_ls = []
emails_ls = []
genders_ls = []
countries_ls = []
age_ls = []

In [11]:
# Loop through each user and extract relevant details
for user in data['results']:
    
    email = user.get('email', np.nan) # if no value is present then it will impute None there
    emails_ls.append(email)
    
    first_name = user['name'].get('first', np.nan)
    first_name_ls.append(first_name)
    
    last_name = user['name'].get('last', np.nan)
    last_name_ls.append(last_name)
    
    gender = user.get('gender', np.nan)
    genders_ls.append(gender)
    
    country = user['location'].get('country', np.nan)
    countries_ls.append(country)
    
    age = user['dob'].get('age', np.nan)
    age_ls.append(age)
    

In [12]:
df = pd.DataFrame({
    'first_name': first_name_ls,
     'last_name': last_name_ls,
     'Email': emails_ls,
     'Gender': genders_ls,
     'Country': countries_ls,
     'age': age_ls
})


In [13]:
df.head()

,first_name,last_name,Email,Gender,Country,age
0,Mattie,Reed,mattie.reed@example.com,female,Australia,47
1,Vanity,De Hoogh,vanity.dehoogh@example.com,female,Netherlands,60
2,Mercedes,Cortes,mercedes.cortes@example.com,female,Spain,42
3,Irma,Luna,irma.luna@example.com,female,Mexico,26
4,Seb,Bloemen,seb.bloemen@example.com,male,Netherlands,66


## Load

create a schema and empty table in sql database before this step:

```create schema users;```

```
create table users.user_details 
(

first_name varchar(500),
last_name varchar(500),
Email varchar(500),
Gender varchar(100),
Country varchar(500),
age varchar(100)
)
```


In [14]:
import mysql.connector
mydb= mysql.connector.connect(host='localhost',user='root',passwd='gunjan', database = 'users')

In [15]:
cursordb= mydb.cursor()

In [16]:
for index, row in df.iterrows(): # it will fetch one row at a time
    
    sql_query = "insert into user_details (first_name,last_name,Email,Gender,Country,age) values (%s,%s,%s,%s,%s,%s)"
    cursordb.execute(sql_query, (row['first_name'],row['last_name'],row['Email'],row['Gender'],row['Country'],row['age']))
print("data loaded successfully")

data loaded successfully


### Create stored procedure in sql to clean the data

```
DELIMITER // 
-- Provide name to procedure
CREATE PROCEDURE users.user_data_transformation ()
BEGIN
drop table if exists users.user_transformed;
create table users.user_transformed
SELECT first_name, last_name, email,
 SUBSTRING(email, LOCATE('@', email) + 1) AS hostname,
 gender, country, 
case when age<=18 then 'below 18'
when age>18 and age<=35 then '18-35'
when age>35 and age<=65 then '35-65'
when age>=65 then 'above 65'
else 'below 18'
end as age_bucket
  FROM   users.user_details; 
END // 
DELIMITER ;
```

In [17]:
# calling stored procedure
cursordb.callproc('user_data_transformation')

()

In [18]:
mydb.commit() # your changes will be reflected after commit
mydb.close()

## now use vscode to create .py file

**step 1** empty the tables,  truncate the users table and drop the output table of stored procedure
<br>**step 2** copy the same code in vscode, also add print and time sleep
<br> **step 3** open the cmd 
<br> **step 4** change directory to D if required using command
```D:```
<br> **step 5** change location cd Documents\DAS\KRPRO_Python\24012024\Projects\ETL
<br> **step 6** using ```dir``` you can see contents in folder
<br> **step 7** please install the python and respective libraries, pandas for eg: ```pip install requests```
<br> **step 8** now run the script using python api_script.py